Too slow actually and not finished

In [26]:
import sys
sys.path.append('../')

import os
import copy
from typing import Tuple, Union, List
import pandas as pd
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm as tqdm
from sklearn.neighbors import NearestNeighbors

import torch
import torch.nn as nn

import torch_geometric as pyg
from torch_geometric.utils.convert import from_networkx

from src.utils import *
from dataset import *
from src.train import train, test
from src.model import BaseEstimator

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from IPython.display import clear_output

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
G = nx.readwrite.read_gpickle('data/network128.gpickle')

In [28]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [29]:
# device = torch.device('cpu')

In [30]:
# Pytorch geometric Data object. For now used only for storing node embedding. 
# Supposed to be used in the future for obtaining node embeddings.
pyg_graph = from_networkx(G)
pyg_graph.to(device)

Data(x=[60789, 128], edge_index=[2, 151294], lng=[60789], lat=[60789], id=[60789], dist=[151294])

$$T(a) = \sum_{b \neq a,\space b\subset Train } T(b)w(a, b)$$

$$ w(a,b)= k \frac{u(a,b)}{\sum_{b \neq a} u(a, b)}  + (1-k) \frac{g(a,b)}{\sum_{b \neq a} g(a,b)};$$

$u(a, b) = exp(-\lambda_1 d(a, b));$

$g(a, b) = exp(-\lambda_2 similarity(x_a, x_b))$, where similarity is cosine distance (can be dot product by the way);

$x_a$ is an embedding for a node **a**;

So for this model $k$, $\lambda_1$, $\lambda_2$ are optimized.

In [31]:
def weight_fn(dists, lamb):
    return torch.exp(-lamb * dists)


class Estimator(BaseEstimator):
    def __init__(self, pyg_graph: pyg.data.Data, observations: Tuple[List, List]) -> None:
        super().__init__(pyg_graph, observations)

        self.k = nn.Parameter(torch.rand(1))
        # self.k = torch.tensor([1.0]).to(device)
        self.lambda_1 = nn.Parameter(torch.rand(1))
        self.lambda_2 = nn.Parameter(torch.rand(1))

    def forward(self, X):
        X_indices = torch.as_tensor(self.node_to_gidx(X.detach().cpu()))
    
        X_embeds = self.g.x[X_indices]

        obs_nodes_indices = self.node_to_gidx(self.obs_nodes)

        # making mask for training nodes
        # other_indices_mask = torch.ones((X.shape[0], self.g.x.shape[0]), dtype=torch.bool)
        # other_indices_mask.scatter_(1, X_indices[:, None], value=False)

        # tmp = self.g.x[None, self.obs_nodes].expand(X.shape[0], -1, -1)
        # other_embeds = tmp[other_indices_mask].reshape(X.shape[0], -1, self.g.x.shape[1])
        similarities = nn.functional.cosine_similarity(X_embeds[:, None], self.g.x[obs_nodes_indices], dim=2)

        # print(similarities.shape)

        # # computing dists
        # tmp = self.g.lng[None, :].expand(X.shape[0], -1)
        # other_lngs = tmp[other_indices_mask].reshape(X.shape[0], -1)

        # tmp = self.g.lat[None, :].expand(X.shape[0], -1)
        # other_lats = tmp[other_indices_mask].reshape(X.shape[0], -1)

        # haversine()

        # return

        # dist_weights = weight_fn(dists, self.lambda_1)
        simi_weights = weight_fn(similarities, self.lambda_2)
        # sum normalizization
        # dist_weights = nn.functional.normalize(dist_weights, p=1)
        simi_weights = nn.functional.normalize(simi_weights, p=1)

        # att_weights = self.k * dist_weights + (1 - self.k) * simi_weights
        
        targets = self.obs_targets.to(device)

        # interpolation
        result = torch.sum(simi_weights.mul(targets), dim=-1)

        return result

In [32]:
results = dict()

In [33]:
loss_fn = nn.HuberLoss(delta=20).to(device)

for day in range(1, 2):
    print(f'Day: {day}')
    path = f'datasets/day_{day}.dat'

    try:
        ds = torch.load(path)
    except:
        continue

    train_loader, val_loader, test_loader = make_data_loaders_from_dataset(ds, train_batch_size=64)
    train_batches = [batch for batch in train_loader]
    train_nodes = torch.cat([batch[0] for batch in train_batches])
    train_targets = torch.cat([batch[1] for batch in train_batches])

    model = Estimator(pyg_graph, (train_nodes, train_targets)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)

    best_model = train(model, train_loader, val_loader, loss_fn, optimizer, device, scheduler=scheduler, plotting=True)
    test_loss, test_score = test(best_model, test_loader, loss_fn)

    # for name, param in best_model.named_parameters():
        # print(name, param)

    results[f'day_{day}'] = test_score 
    print(f'Day {day}, Test loss: {test_loss:.4f}, test score: {test_score:.4f}')

Day: 1


687it [02:32,  4.51it/s]


RuntimeError: CUDA out of memory. Tried to allocate 162.32 GiB (GPU 0; 3.95 GiB total capacity; 126.24 MiB already allocated; 1.78 GiB free; 1.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import json

with open('results/gnn0.json', mode='w') as f:
    json.dump(results, f, indent=4)